## Imports and Setup

In [10]:
# from langchain_community.llms import HuggingFaceHub
# from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from Model import InitializeModel
from GenerateUnitTest import InitializeTestChain
from Utilities import *
load_dotenv()
# from huggingface_hub import notebook_login
# notebook_login()
import pandas as pd    
from DB import *

jsonObj = pd.read_json(path_or_buf='humaneval.jsonl', lines=True)
import sys
from io import StringIO
import sys
from io import StringIO

def runCode(code):
    old_stdout = sys.stdout
    redirected_output = sys.stdout = StringIO()
    oldstderr = sys.stderr
    redirected_output2 = sys.stderr = StringIO()
    result = ""
    thread = CustomThread()
    try:
        thread.start()
        exec(code, globals())
        # print("aaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
        result = redirected_output.getvalue()
    except Exception as e:
        # print(repr(e))
        result = repr(e)
    except SystemExit as s:
        # print(repr(s))
        result = redirected_output2.getvalue()
    except KeyboardInterrupt as k:
        result = "timed out"
    thread.stop()
    globals().popitem()
    sys.stdout = old_stdout
    sys.stderr = oldstderr
    # print(result)
    return result

In [ ]:
from time import sleep
import threading
from _thread import interrupt_main

class CustomThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(CustomThread, self).__init__(*args, **kwargs)
        self._stopper = threading.Event()
    
    def stop(self):
        self._stopper.set()

    def stopped(self):
        return self._stopper.is_set()

    def run(self):
        sleep(5)
        while not self.stopped():
            interrupt_main()

## Initialize Agent

In [11]:
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     GPTNeoForCausalLM,
# )
# from tokenizers.implementations import ByteLevelBPETokenizer
# tokenizer = ByteLevelBPETokenizer(
#     "vocab.json",
#     "merges.txt",
# )
load_dotenv()
model_id="google/gemma-7b-it"
jsonObj = pd.read_json(path_or_buf='humaneval.jsonl', lines=True)
llm, chat_model = InitializeModel(os.environ['HF_TOKEN'], model_id, 5000)

GenUnitTestChain= InitializeTestChain(chat_model)
db= connect_db()

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


## Generate Unit Tests from Code

In [ ]:
def get_few_shots(db, code):
    test_cases_of_few_shotList=[]
    description_of_few_shotList=[]
    code_of_few_shotList=[]
    query = code
    docs = db.similarity_search(query)
    # print("def" + docs[0].metadata['test'].split("def", maxsplit = 1)[1])
    # print(docs[3].metadata)
    for i in range(1,4):
        
        test_cases_of_few_shot =docs[i].metadata["test"]
        description_of_few_shot =docs[i].metadata["prompt"]
        code_of_few_shot=docs[i].page_content
        #get the function header 
        code_of_few_shot = re.sub(r"\\n", "\n", code_of_few_shot)
        description_of_few_shot = re.sub(r"\'\'\'", "\"\"\"", description_of_few_shot)
        rs = re.search(r'\"\"\".*\"\"\"', description_of_few_shot, re.DOTALL) 
        end = rs.span()[0]
        description_of_few_shot = description_of_few_shot[:end]
        #print(test_cases_of_few_shot)
        code_of_few_shot = description_of_few_shot + code_of_few_shot
        test_cases_of_few_shotList.append(test_cases_of_few_shot)
        description_of_few_shotList.append(description_of_few_shot)
        code_of_few_shotList.append(code_of_few_shot)
    return (code_of_few_shotList,test_cases_of_few_shotList)

In [ ]:
def preprocessString(code, unittest):
    string_few_shot = ""
    code_prepend = "# Code of a similar function \n'''python\n"
    test_case_prepend = "# Example unit tests for the similar code \n'''python\n"
    code_append = test_case_append = "'''\n"
    for i in zip(code, unittest):
        string_few_shot += code_prepend + i[0] + code_append + test_case_prepend + i[1] + test_case_append
    return string_few_shot

In [12]:
introCode = """
import sys
sys.setrecursionlimit(100)
"""

OKCases=0
descriptions=[]
codes=[]
res_codes=[]
feedbacks=[]

for i in range(0,50):
    # if (i == 10): continue
    print("Running Test Case ",i+1)
    # description and code from database
    description=jsonObj.iloc[i]['text']
    code=jsonObj.iloc[i]['canonical_solution']
    indexer = 0
    while code[indexer] == " ":
        indexer += 1
    code = code[indexer:]
    # get the function header and add it to the code
    funcDefiniton=jsonObj.iloc[i]['prompt']
    rs = re.search(r'\"\"\".*\"\"\"', funcDefiniton, re.DOTALL) 
    end = rs.span()[0]
    funcDefiniton = funcDefiniton[:end]
    code = funcDefiniton + code
    ############################################

    # get the few shot code and test cases
    code_of_few_shot,test_cases_few_shot=get_few_shots(db,code) #changed from invoke to run**********
    fewShotStr = preprocessString(code_of_few_shot, test_cases_few_shot)
    # print("fewShotStr is :",fewShotStr)
    unittest=GenUnitTestChain.invoke({"description":description,"code":code})
    ############################################
    ### get the response (unit test) code only without natural language
    unittest_code = get_code_from_response(unittest['text'])
    unittest_code = replaceUnitTestCall(unittest_code)

    ##############run code with the original and unit test concatenated #######################
    codeTobeRun = introCode + "\n" + code + "\n" + unittest_code 
    print("run_code is")
    print(codeTobeRun)
    # print("------------------------------lolooloolololo ------------------------------------")
    feedback = runCode(code=codeTobeRun)
    feedbackparsed=get_feedback_from_run(feedback)
    if(feedbackparsed==''):
        #in case of OK
        OKCases+=1
        # feedbackparsed=""
        # for i,line in enumerate(feedback.split('\n')):
        #     if i>1:
        #         feedbackparsed+=line+'\n'
    #print(feedbackparsed)
    print(feedback)
    descriptions.append(description)
    codes.append(code)
    res_codes.append(unittest_code)
    feedbacks.append(feedbackparsed)  #feedbackparsed

#dump data into xlsx file
resDF=pd.DataFrame({'Description':descriptions,'Code':codes,'GeneratedCode':res_codes,'Feedback':feedbacks})
# resDF.to_excel("GeneratedFeedback.xlsx",index=False)
resDF.to_json("GeneratedFeedback.json",orient='records')
print("OKCases are ",OKCases)
print("Total Cases are ",len(descriptions))
with open('Res.txt', 'w') as f:
    f.write("OKCases are "+str(OKCases)+"\n")
    f.write("Total Cases are "+str(len(descriptions))+"\n")


Running Test Case  1
description is
    Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
code is
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False



> Entering new LLMChain chain...
Prompt after formatting:
Given the following description and python code:
Description:
    Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
Code:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
           

## Getting HumanEval

In [ ]:
from langchain_community.document_loaders import JSONLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import HuggingFaceDatasetLoader

dataset_name = "openai_humaneval"
page_content_column = "canonical_solution"
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)
data = loader.load()
db = Chroma.from_documents(data, embedding_function)

KeyboardInterrupt: 

In [ ]:
query = code
docs = db.similarity_search(query)
print(docs[0].metadata['test'])
test_cases_of_few_shot=docs[0].metadata['test']

def check(candidate):

    # Check some simple cases
    assert candidate('[[]]') == True, "This prints if this assert fails 1 (good for debugging!)"
    assert candidate('[]]]]]]][[[[[]') == False
    assert candidate('[][]') == False
    assert candidate(('[]')) == False
    assert candidate('[[[[]]]]') == True
    assert candidate('[]]]]]]]]]]') == False
    assert candidate('[][][[]]') == True
    assert candidate('[[]') == False
    assert candidate('[]]') == False
    assert candidate('[[]][[') == True
    assert candidate('[[][]]') == True

    # Check some edge cases that are easy to work out by hand.
    assert candidate('') == False, "This prints if this assert fails 2 (also good for debugging!)"
    assert candidate('[[[[[[[[') == False
    assert candidate(']]]]]]]]') == False




## Generate Unit Tests using Few Shots examples

In [ ]:
description="""def is_nested(string): ''' Create a function that takes a string as input which contains only square brackets. The function should return True if and only if there is a valid subsequence of brackets where at least one bracket in the subsequence is nested. is_nested('[[]]') ➞ True is_nested('[]]]]]]][[[[[]') ➞ False is_nested('[][]') ➞ False is_nested('[]') ➞ False is_nested('[[][]]') ➞ True is_nested('[[]][[') ➞ True '''"""
code="""def is_nested(string):
    opening_bracket_index = []
    closing_bracket_index = []
    for i in range(len(string)):
        if string[i] == '[':
            opening_bracket_index.append(i)
        else:
            closing_bracket_index.append(i)
    closing_bracket_index.reverse()
    cnt = 0
    i = 0
    l = len(closing_bracket_index)
    for idx in opening_bracket_index:
        if i < l and idx < closing_bracket_index[i]:
            cnt += 1
            i += 1
    return cnt >= 2"""

Gen_UnitTest_with_FewShots_template="""Task: Given the following description and python code:

Description:
{description}
Code:
'''python
{code}'''

Generate a class that contains multiple unit tests written in python to find bugs, syntax errors or logical errors in the code to conform with the description and include any required imports.
You are given example unit tests of a similar code from a database:
'''python
{test_cases_of_few_shot}'''


Generate a class that contains multiple unit tests written in python to find bugs, syntax errors or logical errors in the code to conform with the description and include any required imports.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```python" and "```":"""
# Generate multiple unit tests in python code in form of functions to find bugs, syntax errors or logical errors and if the code functionality does not comply with the description.

Generate_Unit_Tests_FewShots__Template = PromptTemplate(
    template=Gen_UnitTest_with_FewShots_template,
    input_variables=["description","code","test_cases_of_few_shot"],
    verbose=True
    )

GenUnitTestChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=Generate_Unit_Tests_FewShots__Template
)
res=GenUnitTestChain.invoke({
                             "description":description,
                             "code":code,
                             "test_cases_of_few_shot":test_cases_of_few_shot
                             })
print(res['text'])



> Entering new LLMChain chain...
Prompt after formatting:
Task: Given the following description and python code:

Description:
def is_nested(string): ''' Create a function that takes a string as input which contains only square brackets. The function should return True if and only if there is a valid subsequence of brackets where at least one bracket in the subsequence is nested. is_nested('[[]]') ➞ True is_nested('[]]]]]]][[[[[]') ➞ False is_nested('[][]') ➞ False is_nested('[]') ➞ False is_nested('[[][]]') ➞ True is_nested('[[]][[') ➞ True '''
Code:
'''python
def is_nested(string):
    opening_bracket_index = []
    closing_bracket_index = []
    for i in range(len(string)):
        if string[i] == '[':
            opening_bracket_index.append(i)
        else:
            closing_bracket_index.append(i)
    closing_bracket_index.reverse()
    cnt = 0
    i = 0
    l = len(closing_bracket_index)
    for idx in opening_bracket_index:
        if i < l and idx < closing_bracket_index[i

In [ ]:
#made according to mixtral response
def get_code_from_response(response):
    lines=response.split("\n")
    code=""
    in_code=False
    in_new_response=False
    for i,line in enumerate(lines):
        if in_new_response:
          if line.startswith("```python"):
            in_code=True
          elif line.startswith("```"):
            in_code=False
            return code
          elif in_code==True:
            if(i==len(lines)-1):
              return code
            code+=line+"\n"
        else:
            if "[/INST]" in line:
                in_new_response=True
                # case where the llm only wrote a code instead of a full response with other text
                if "```python" in line :
                    in_code=True
    #incomplete output, most likely incomplete assertion.
    # ignore this last assertion and move with the rest

    return code
# replaces the unittest call with another one to fix exec problem with running on colab
def replaceUnitTestCall(code):
    pattern = r"unittest.main()"

    # Use re.sub() to replace the pattern with the replacement string
    modified_code = re.sub(pattern, "unittest.main(argv=['first-arg-is-ignored'])", code)

    return modified_code



res_code= get_code_from_response(res['text'])
res_code= replaceUnitTestCall(res_code)
print("res_code is")
print(res_code)

res_code is
import unittest

class TestIsNested(unittest.TestCase):

    def test_simple_cases(self):
        self.assertTrue(is_nested('[[]]'))
        self.assertFalse(is_nested('[]]]]]]][[[[[]'))
        self.assertFalse(is_nested('[][]'))
        self.assertFalse(is_nested('[]'))
        self.assertTrue(is_nested('[[[[]]]]'))
        self.assertFalse(is_nested('[]]]]]]]]]]'))
        self.assertTrue(is_nested('[][][[]]'))
        self.assertFalse(is_nested('[[]'))
        self.assertFalse(is_nested('[]]'))
        self.assertTrue(is_nested('[[]][['))
        self.assertTrue(is_nested('[[][]]'))

    def test_edge_cases(self):
        self.assertFalse(is_nested(''))
        self.assertFalse(is_nested('[[[[[[[['))
        self.assertFalse(is_nested(']]]]]]]]'))

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'])()



In [ ]:
print(runCode(code+"\n"+res_code))

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK



In [8]:
code="""def is_nested(string):
    opening_bracket_index = []
    closing_bracket_index = []
    for i in range(len(string)):
        if string[i] == '[':
            opening_bracket_index.append(i)
        else:
            closing_bracket_index.append(i)
    closing_bracket_index.reverse()
    cnt = 0
    i = 0
    l = len(closing_bracket_index)
    for idx in opening_bracket_index:
        if i < l and idx < closing_bracket_index[i]:
            cnt += 1
            i += 1
    return cnt >= 2"""
falsecode="""import unittest

class TestIsNested(unittest.TestCase):

    def test_simple_cases(self):
        self.assertTalse(is_nested('[[]]'))
        self.assertFalse(is_nested('[]]]]]]][[[[[]'))
        self.assertFalse(is_nested('[][]'))
        self.assertFalse(is_nested('[]'))
        self.assertTrue(is_nested('[[[[]]]]'))
        self.assertFalse(is_nested('[]]]]]]]]]]'))
        self.assertTrue(is_nested('[][][[]]'))
        self.assertFalse(is_nested('[[]'))
        self.assertFalse(is_nested('[]]'))
        self.assertTrue(is_nested('[[]][['))
        self.assertTrue(is_nested('[[][]]'))

    def test_edge_cases(self):
        self.assertTrue(is_nested(''))
        self.assertFalse(is_nested('[[[[[[[['))
        self.assertFalse(is_nested(']]]]]]]]'))

if __name__ == '__main__':
    # unittest.main(argv=['first-arg-is-ignored' ])()
    # unittest.main()
"""
from Utilities import *
feedback=runCode(code+"\n"+falsecode)
print(feedback)

## Bug Fixing

In [ ]:
def get_feedback_from_run(response):
  lines=response.split("\n")
  feedback=""
  in_failmessage=False
  for i,line in enumerate(lines):
      if line.startswith("FAIL") or line.startswith("ERROR"):
        in_failmessage=True
        feedback+=line+"\n"
      elif line.startswith("--"):
        if lines[i+1].startswith("Ran"):
            return feedback

      elif in_failmessage==True:
        if line.startswith("=="):
            continue
        if(i == len(lines)-1):
          return feedback
        feedback += line + "\n"

def get_feedback_from_run_list(response):  # version omar
    lines=response.split("\n")
    in_failmessage = False
    feedbacks = []
    for i,line in enumerate(lines):
        if line.startswith("FAIL") or line.startswith("ERROR"):
          feedback=""
          in_failmessage=True
          feedback+=line+"\n"
        elif line.startswith("--"):
          if lines[i+1].startswith("Ran"):
              if (feedback != ""):
                feedbacks.append(feedback)
              return feedbacks

        elif in_failmessage==True:
          if line.startswith("=="):
              if (feedback != ""):
                feedbacks.append(feedback)
                feedback = ""
                continue
          if(i == len(lines)-1):
            if (feedback != ""):
                feedbacks.append(feedback)
            return feedbacks
          feedback += line + "\n"

# resultO = get_feedback_from_run_list(feedback)
# print(resultO)
# feedbackparsed = get_feedback_from_run(feedback)
# # redundant but OK, we already know that it is empty and it is a sufficient flag
# if(feedbackparsed==''):
#     #in case of OK
#     feedbackparsed=""
#     for i,line in enumerate(feedback.split('\n')):
#         if i>1:
#             feedbackparsed+=line+'\n'

# print(feedbackparsed)

In [ ]:
BugRepair_template_str = """Act as a python expert in bug fixing and assume that I need to fix all errors perfectly.
I am going to give you the bugs encountered due to running test cases then after
that I will give you the code under test with bugs so that you fix it, the line numbers of
the error correspond to the line that the error occured in the code provided.
The following is the code under test:
```python
{code}```

The following are the bugs and test cases corresponding to the bugs:
```python
{TestcaseCode}```

The result after running these test cases on an interpreter:
{InterpreterFeedback}

Change either the code or the test cases according to the feedback of the interpreter.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```python" and "```":
"""
InterpreterFeedback=feedbackparsed

BugRepair_Template = PromptTemplate(
    template=BugRepair_template_str,
    input_variables=["code","TestcaseCode","InterpreterFeedback"],
    verbose=True
    )

BugFixChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=BugRepair_Template
)

res=BugFixChain.invoke({
                             "code":code,
                             "TestcaseCode":res_code,
                             "InterpreterFeedback":InterpreterFeedback
                             })
print(res['text'])



> Entering new LLMChain chain...
Prompt after formatting:
Act as a python expert in bug fixing and assume that I need to fix all errors perfectly.
I am going to give you the bugs encountered due to running test cases then after
that I will give you the code under test with bugs so that you fix it, the line numbers of 
the error correspond to the line that the error occured in the code provided.
The following is the code under test:
```python
def is_nested(string):
    opening_bracket_index = []
    closing_bracket_index = []
    for i in range(len(string)):
        if string[i] == '[':
            opening_bracket_index.append(i)
        else:
            closing_bracket_index.append(i)
    closing_bracket_index.reverse()
    cnt = 0
    i = 0
    l = len(closing_bracket_index)
    for idx in opening_bracket_index:
        if i < l and idx < closing_bracket_index[i]:
            cnt += 1
            i += 1
    return cnt >= 2```

The following are the bugs and test cases corresponding

## input code preprocessor

In [ ]:
#if the input code is without description, we want to generate description
Generate_description_Template = """Act as a python expert, Describe the purpose and functionality of the following Python code.

```python
{code}```
The output should be like that:
The provided Python code aims to [explain purpose].
In summary, [concise summary of functionality ].
Examples: [3 or 4 input output examples]
"""
code="""def is_nested(string):
    opening_bracket_index = []
    closing_bracket_index = []
    for i in range(len(string)):
        if string[i] == '[':
            opening_bracket_index.append(i)
        else:
            closing_bracket_index.append(i)
    closing_bracket_index.reverse()
    cnt = 0
    i = 0
    l = len(closing_bracket_index)
    for idx in opening_bracket_index:
        if i < l and idx < closing_bracket_index[i]:
            cnt += 1
            i += 1
    return cnt >= 2"""
Generate_description_PromptTemplate = PromptTemplate(
    template=Generate_description_Template,
    input_variables=["code"],
    verbose=True
    )

GenDescriptionChain = LLMChain(
    llm=chat_model,
    verbose=True,
    prompt=Generate_description_PromptTemplate
)
res=GenDescriptionChain.invoke({
                             "code":code
                             })
print(res['text'])